# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yarmouth,41.7057,-70.2286,5.56,90,100,8.05,US,1733424941
1,1,tsiombe,-25.3000,45.4833,24.84,88,0,2.31,MG,1733424942
2,2,hermanus,-34.4187,19.2345,18.22,73,100,7.91,ZA,1733424943
3,3,tamanrasset,22.7850,5.5228,15.95,13,0,2.06,DZ,1733424944
4,4,tiksi,71.6872,128.8694,-24.96,95,38,3.12,RU,1733424945


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    "Lat",
    geo = True,
    tiles = 'OSM',
    frame_width=1000,
    frame_height=600,
    size = 'Humidity',
    color = 'City')


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 22) & 
    (city_data_df['Humidity'] < 80) & 
    (city_data_df['Cloudiness'] < 40) & 
    (city_data_df['Wind Speed'] < 10)
].dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,afonso claudio,-20.0742,-41.1239,26.97,67,6,2.63,BR,1733424946
7,7,quatre cocos,-20.2078,57.7625,25.06,71,25,5.45,MU,1733424948
8,8,tuxpan,19.5500,-103.4000,29.70,22,21,1.72,MX,1733424949
17,17,nova sintra,14.8667,-24.7167,26.05,66,1,2.54,CV,1733424959
28,28,natal,-5.7950,-35.2094,29.36,61,20,7.20,BR,1733424740
...,...,...,...,...,...,...,...,...,...,...
539,539,ceelbuur,4.6850,46.6176,25.93,58,12,5.47,SO,1733425540
555,555,broken hill,-31.9500,141.4333,29.87,37,26,5.08,AU,1733425352
556,556,ormara,25.2088,64.6357,24.97,33,39,6.40,PK,1733425334
564,564,julio de castilhos,-29.2269,-53.6817,30.26,42,6,1.42,BR,1733425529


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,afonso claudio,BR,-20.0742,-41.1239,67,
7,quatre cocos,MU,-20.2078,57.7625,71,
8,tuxpan,MX,19.5500,-103.4000,22,
17,nova sintra,CV,14.8667,-24.7167,66,
28,natal,BR,-5.7950,-35.2094,61,
...,...,...,...,...,...,...
539,ceelbuur,SO,4.6850,46.6176,58,
555,broken hill,AU,-31.9500,141.4333,37,
556,ormara,PK,25.2088,64.6357,33,
564,julio de castilhos,BR,-29.2269,-53.6817,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
afonso claudio - nearest hotel: Hotel Custódio
quatre cocos - nearest hotel: Horisun
tuxpan - nearest hotel: Hotel Ruiz
nova sintra - nearest hotel: Residência Ka Dencho
natal - nearest hotel: Natal Palace Hotel
kavaratti - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
antsiranana - nearest hotel: Central
puerto san jose - nearest hotel: Martita
mbaiki - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
acapulco de juarez - nearest hotel: Hotel del Valle
pisco - nearest hotel: La Portada
isla aguada - nearest hotel: Puerto Sabalo Hotel
kismayo - nearest hotel: Kismayo Hotel
valente - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
tolanaro - nearest hotel: Hôtel Mahavokey
san julian - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
5,afonso claudio,BR,-20.0742,-41.1239,67,Hotel Custódio
7,quatre cocos,MU,-20.2078,57.7625,71,Horisun
8,tuxpan,MX,19.5500,-103.4000,22,Hotel Ruiz
17,nova sintra,CV,14.8667,-24.7167,66,Residência Ka Dencho
28,natal,BR,-5.7950,-35.2094,61,Natal Palace Hotel
...,...,...,...,...,...,...
539,ceelbuur,SO,4.6850,46.6176,58,No hotel found
555,broken hill,AU,-31.9500,141.4333,37,Ibis Styles
556,ormara,PK,25.2088,64.6357,33,No hotel found
564,julio de castilhos,BR,-29.2269,-53.6817,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    'Lng',
    "Lat",
    geo = True,
    tiles = 'OSM',
    frame_width=1000,
    frame_height=600,
    size = 'Humidity',
    color = 'City',
    hover_cols=['City', 'Hotel Name', 'Country'])

# Display the map
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)